## Defining models and their field types with Pydantic

### Standard field types

In [1]:
import time

from pydantic import BaseModel


class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [3]:
from datetime import date
from enum import Enum
from typing import List

from pydantic import BaseModel, ValidationError


class Gender(str, Enum):
    MALE = "MALE"
    FEMALE = "FEMALE"
    NON_BINARY = "NON_BINARY"


class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    gender: Gender
    birthday: date
    interests: List[str]

In [5]:
# invalid gender

try:
    p = Person(first_name="John", last_name='Doe', age=14, gender='INVALID_VALUE', birthday='1991-01-01',
               interests=['sport', 'reading'])
except ValidationError as e:
    print(str(e))

1 validation error for Person
gender
  value is not a valid enumeration member; permitted: 'MALE', 'FEMALE', 'NON_BINARY' (type=type_error.enum; enum_values=[<Gender.MALE: 'MALE'>, <Gender.FEMALE: 'FEMALE'>, <Gender.NON_BINARY: 'NON_BINARY'>])


In [6]:
from datetime import date

my_date = date.today()
print(my_date.isoformat())

2021-12-19


In [7]:
try:
    p = Person(first_name="John", last_name='Doe', age=14, gender='MALE', birthday='1991-13-01',
               interests=['sport', 'reading'])
except ValidationError as e:
    print(str(e))

1 validation error for Person
birthday
  invalid date format (type=value_error.date)


In [9]:
try:
    p = Person(first_name="John", last_name='Doe', age=14, gender='MALE', birthday='1991-11-01',
               interests=['sport', 'reading'])
except ValidationError as e:
    print(str(e))

In [10]:
p

Person(first_name='John', last_name='Doe', age=14, gender=<Gender.MALE: 'MALE'>, birthday=datetime.date(1991, 11, 1), interests=['sport', 'reading'])

In [13]:
class Address(BaseModel):
    street_address: str
    postal_code: str
    city: str
    country: str


class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    gender: Gender
    birthday: date
    address: Address

In [14]:
try:
    p = Person(first_name="John", last_name='Doe', age=14, gender='MALE', birthday='1991-11-01',
               address={
                   "street_address": 'gabdullin',
                   'postal_code': '050040',
                   'city': 'Almaty',
                   'country': 'KZ'
               })
except ValidationError as e:
    print(str(e))

In [15]:
p

Person(first_name='John', last_name='Doe', age=14, gender=<Gender.MALE: 'MALE'>, birthday=datetime.date(1991, 11, 1), address=Address(street_address='gabdullin', postal_code='050040', city='Almaty', country='KZ'))

### Optional fields and default values

In [16]:
from typing import Optional
from pydantic import BaseModel


class UserProfile(BaseModel):
    nickname: str
    location: Optional[str] = None
    subscribed_newsletter: bool = True


user = UserProfile(nickname='Joe')

In [17]:
user

UserProfile(nickname='Jdoe', location=None, subscribed_newsletter=True)

In [19]:
from datetime import datetime


class Model(BaseModel):
    # Don;t do this, this example shows you why it doesn't work
    d: datetime = datetime.now()


o1 = Model()

In [20]:
o1

Model(d=datetime.datetime(2021, 12, 19, 18, 49, 44, 394293))

In [22]:
import time

time.sleep(1)

In [23]:
o2 = Model()

In [24]:
o2

Model(d=datetime.datetime(2021, 12, 19, 18, 49, 44, 394293))

In [25]:
print(o1.d < o2.d)

False


Even though we waited for 1 second between the instantiation of o1 and o2, the d datetime is the same!

 Pydantic provides a Fieldfunction that allows us to set some advanced options on our fields, including one to set a factory for creating dynamic values.

### Field validation

In [26]:
from typing import Optional

from pydantic import BaseModel, Field, ValidationError


class Person(BaseModel):
    first_name: str = Field(..., min_length=3)
    last_name: str = Field(..., min_length=3)
    age: Optional[int] = Field(None, ge=0, le=150)

The first positional argument defines the default value for the field. If the field is required, we use the ellipsis .... Then, the keyword arguments are there to set options for the field, including some basic validation

### Dynamic default values

In [28]:
from datetime import datetime
from typing import List

from pydantic import BaseModel, Field


def list_factory():
    return ["a", "b", "c"]


class Model(BaseModel):
    l: List[str] = Field(default_factory=list_factory)
    d: datetime = Field(default_factory=datetime.now)
    l2: List[str] = Field(default_factory=list)

This argument expects you to pass a function that will be called during model instantiation. Thus, the resulting object will be evaluated at runtime each time you create a new object.

In [29]:
m = Model()

m

In [30]:
m

Model(l=['a', 'b', 'c'], d=datetime.datetime(2021, 12, 19, 18, 59, 1, 890841), l2=[])

In [31]:
m1 = Model()

In [32]:
m1

Model(l=['a', 'b', 'c'], d=datetime.datetime(2021, 12, 19, 18, 59, 26, 73980), l2=[])

## Validating email address and URLs with Pydantic types

In [33]:
!pip install email-validator

In [34]:
from pydantic import BaseModel, EmailStr, HttpUrl, ValidationError


class User(BaseModel):
    email: EmailStr
    website: HttpUrl


In [35]:
try:
    User(email="jon", website='https://azat.ai')
except ValidationError as e:
    print(e)

1 validation error for User
email
  value is not a valid email address (type=value_error.email)


In [36]:
try:
    User(email="jon@gmail.", website='https://azat.ai')
except ValidationError as e:
    print(e)

1 validation error for User
email
  value is not a valid email address (type=value_error.email)


In [37]:
try:
    User(email="jon@azat.ai", website='https://azat.ai')
except ValidationError as e:
    print(e)

In [38]:
try:
    User(email="jon@azat.ai", website='https://azat')
except ValidationError as e:
    print(e)

1 validation error for User
website
  URL host invalid, top level domain required (type=value_error.url.host)


## Creating Model Variations with Class Inheritance

In [39]:
from pydantic import BaseModel


class PostCreate(BaseModel):
    title: str
    content: str


class PostPublic(BaseModel):
    id: int
    title: str
    content: str


class PostDB(BaseModel):
    id: int
    title: str
    content: str
    nb_views: int = 0

PostCreate will be used for a POST endpoint to create a new post. We expect the user to give the title and the content; however, the identifier(ID) will be automatically determined by the database

PostPublic will be used when we retrieve the data of a post. We want its title and content, of course, but also its associated ID in the database

PostDB will carry all the data we wish to store in the database. Here, we also want to store the number of views, but we want to keep this secret to make our own statistics internally

In [41]:
from pydantic import BaseModel


class PostBase(BaseModel):
    title: str
    content: str


class PostCreate(PostBase):
    pass


class PostDB(PostBase):
    id: int
    views: int = 0


class PostPublic(PostBase):
    id: int

It's also very convenient if you wish to define methods on your model. Remember that Pydantic models are regular Python classes, so you can implement as many methods as you wish!

In [42]:
class PostBase(BaseModel):
    title: str
    content: str

    def excerpt(self) -> str:
        return f"{self.content[:140]}"

## Adding Custom Data Validation With Pydantic

### Applying validation at a field level
